In [ ]:
def paso_intpoint(u,v):
    # Recorte de paso para el método de puntos interiores
    # para Programación Cuadrática.
    # u es un vector de dimensión p tal que u[i]>0 para todo i
    # v es un vector de dimensión p.
    # Construir un escalar alfa <= 1 tal que
    # u +(alfa)*v >=0.
    # Si v[i] >= 0 para toda i
    # el escalar alfa = 1.
    # En caso contrario existe un índice i tal que v[i] <0.
    #------------------------------------------------
    # Optimización Numérica
    #   ITAM
    #    20 de marzo de 2024
    #-----------------------------------------
    import numpy as np
    p = len(u)
    v_alfa = np.ones(p)
    for i in range(p):
         if(v[i]<0):
             v_alfa[i] = -u[i]/v[i]
             #---Fin de if---
             
     #----------Fin de for -----------

    alfa = np.amin(v_alfa)
    alfa = np.amin([alfa, 1.0])        
    return alfa
    #------- Fin de paso:intpoint.py-----------------

In [ ]:
def myqp_intpoint(Q, A, F,c,b,d):
    # Método de Puntos Interiores para Programación
    # Cuadrática
    # Min (1/2) x.T*Q*x + c.T*x
    # s.a.   A*x = b
    #        F*x >= d
    # Q es simétrica positiva definda de orden n
    # A es matriz mxn tal que rango(A) = m.
    # F matriz pxn.
    # c vector de dimensión n
    # b vector de dimensión m
    # d vector de dimensión 
    #
    # return
    # x vector de dimensión n con el mínimo del problema
    # y vector de dimensión m y es el multiplicador de
    #    Lagrange para: A*x . b = 0
    # mu vector de dimensión p y es el multiplicador de Lagrange
    #    para_ f*x -d >= 0
    # z vector de dimensión p tal que z = F*x-d 
    # iter número de iteraciones que usamos
    #-----------------------------------------------
    # Optimazación Numérica
    #   ITAM
    # 20 de marzo de 2024
    #-----------------------------------------------
    import numpy as np
    import copy
    from myqp import paso_intpoint
    
    #---------Valores Iniciales---------
    n = len(c)
    m = len(b)
    p = len(d)
    tol = 10**(-5)
    maxiter = 100
    iter = 0
    #-------------Vectores iniciales-----------
    x = np.ones(n)
    y = np.zeros(m)
    mu= np.ones(p)
    z = np.ones(p)
    #--------------------------------
    # lado derecho
    tau = 1/2
    v1 = np.dot(Q,x)+np.dot(A.T, y)-np.dot(F.T,mu) +c
    v2 = np.dot(A,x)-b
    v3 = -np.dot(F,x) +z +d
    v4 = np.multiply(mu,z)

    cnpo = np.concatenate((v1,v2),0)
    cnpo = np.concatenate((cnpo, v3),0)
    cnpo = np.concatenate((cnpo, v4),0)
    norma_cnpo = np.linalg.norm(cnpo)
    #--------------------------------------------
    # Proceso iterativo
    while(norma_cnpo > tol and iter < maxiter):
        cnpo_pert = copy.copy(cnpo)
        cnpo_pert[n+m+p:n+m+p+p]= cnpo_pert[n+m+p:n+m+p+p]-tau
        #---------------------------------------
        #  Matriz de Newton
        dim = n + m + p +p
        M = np.zeros((dim,dim))
        M[0:n, 0:n]  = Q
        M[0:n, n:(n+m)]= A.T
        M[0:n, (n+m):(n+m+p)]=-F.T
        M[n:n+m, 0:n]= A
        M[(n+m):(n+m+p), 0:n]= -F
        M[(n+m):(n+m+p), (n+m+p):(n+m+p+p)]= np.identity(p)
        M[(n+m+p):dim, (n+m):(n+m+p)]= np.diag(z)
        M[(n+m+p):dim, (n+m+p):dim]= np.diag(mu)
        #-------------------------------------
        # Solución del sistema lineal
        dw = np.linalg.solve(M,-cnpo_pert)
        dx = dw[0:n]
        dy = dw[n:n+m]
        dmu =dw[n+m:n+m+p]
        dz =dw[n+m+p:dim]
        #-----------------------------------
        alfa1 = paso_intpoint(mu,dmu)
        alfa2 = paso_intpoint(z,dz)
        alfa = (0.95)*np.amin([alfa1, alfa2, 1.0])
        #---------------------------------
        # Actualizamos los vectores-----
        x =  x  + alfa*dx
        y =  y  + alfa*dy
        mu = mu + alfa*dmu
        z =  z  + alfa*dz
        #--------------------------------
        iter = iter +1
        tau = np.dot(mu,z)/(2*p)
        #------------------------------------
        v1 = np.dot(Q,x)+np.dot(A.T, y)-np.dot(F.T,mu) +c
        v2 = np.dot(A,x)-b
        v3 = -np.dot(F,x) +z +d
        v4 = np.multiply(mu,z)

        cnpo = np.concatenate((v1,v2),0)
        cnpo = np.concatenate((cnpo, v3),0)
        cnpo = np.concatenate((cnpo, v4),0)
        norma_cnpo = np.linalg.norm(cnpo)
        
        print("iter=", iter,"|","||cnpo||=",norma_cnpo)
        #----------------------------------
        if(norma_cnpo <=tol or iter ==maxiter):
           return x,y,mu,z,iter
           break
       
        #  ---- Fin de myqp_intpoint.py----
        #------------------------------------